# TME 5 : Localisation de pattern (motifs) dans le genome

Nous allons implémenter plusieurs algorithmes afin de localiser un motif d'intérêt dans un génome.

### A -  Index of fixed length words
3\. Nous allons implémenter l'algorithme "Index of fixed length words". Faire une fonction pour indexer les positions d'occurrences de tous les mots de taille k dans un texte (génome), la fonction doit renvoyer un dictionnaire ou les clés sont les mots et les valeurs les positions dans le texte où se trouve les mots.

In [ ]:
def indexTable(m, sequence):
    """
    Indexer les positions d'occurrences de tous les mots de taille k dans une sequence
    entrée m : taille du mot à chercher dans le motif m <= k
    entrée sequence : chaine de caractère représentant une sequence d'ADN
    sortie indexes : dictionaire où les clés sont les mots et les valeurs les positions dans la sequence
    """
    indexes  = {}

    return indexes

inds = indexTable(2, "abcdabda") #{'ab': [0, 4], 'bc': [1], 'cd': [2], 'da': [3, 6], 'bd': [5]}
print (inds)

4\. Faire une fonction pour chercher toutes les occurrences d'un motif dans un génome utilisant la table des indexes de la question précédente. Nous allons permettre un nombre maximum de variations. Pour cela, implémenter ou re-utiliser la fonction distance de hamming

In [ ]:
def hamming(seq1, seq2):
    count = 0
    
    return count


def chercherWithIndexTable(m, table, sequence, motif, maxVar):
    """
    chercher les positions d'un motif dans une séquence en admettant au maximum maxVar variations
    entrée m : taille du mot à chercher dans le motif m <= k, le meme utilise pour indexer les sequences
    entrée table : dictionaire où les clés sont les mots et les valeurs les positions dans la sequence
    entrée sequence : chaine de caractère représentant une sequence d'ADN
    entrée motif : chaine de caractère représentant le motif à chercher
    entrée maxVar : le maximum variations entre le motif et un mot de taille k dans la sequence
    sortie motifPos : dictionnaire où les clés sont les motifs trouvé et les valeurs leurs positions dans la sequence.
    """
    k = len(motif)
    motifPos = {}
    motif = motif.lower()

    return motifPos


m = 3; motif =  "TAGTT"; maxVar = 2
inds = indexTable(m, "ACGAAATAGTTAGAA")
print (inds)
#{'acg': [0], 'cga': [1], 'gaa': [2, 12], 'aaa': [3], 'aat': [4], 'ata': [5], 'tag': [6, 10], 'agt': [7], 'gtt': [8], 'tta': [9], 'aga': [11]}
motifPos = chercherWithIndexTable(m, inds, 'ACGAAATAGTTAGAA', "TAGTT", maxVar)
print (motifPos)
#{'tagtt': [6]}

5\. Tester l'algorithme **"Index of fixed length words"** sur vos données de chipSeq. Pour cela développer une fonction pour chercher un motif d'intérêt sur chaque séquence du fichier Peaks.
Voici les motifs que vous devez trouver selon le fichier de Peaks:

 - Peaks 1 Atf1 (TGCACCC / GGGTGCA)
 - Peaks 2 Atm1 (CAGCAAaaa / tttTTGCTG)
 - Peaks 3 Hap4 (CCAAT / ATTGG)
 - Peaks 4 Mac1 (GAGCAAA / TTTGCTC)
 - Peaks 5 Mac1 (GAGCAAA / TTTGCTC)
 - Peaks 6 Pdr1 (CCACGGA / TCCGTGG)
 - Peaks 7 Pdr1 (CCACGGA / TCCGTGG)

In [ ]:
import time

k = 7
genome = "C_glabrata_1000bp_only.fasta"
peaks = "Sequence_by_Peaks_##.fasta" ### modifier avec votre fichier !!!
motif = 'NNNNN' # Le motif à localiser pour votre fichier, par example "tgcaccc"

# Fonction modifié
def readFasta(fastaFileName):
    """
    Read a fasta file
    entrée fastaFileName: nom du fichier fasta
    sortie sequences: liste contenant toutes les sequences du fichier
    """

    sequence = ""
    sequences_list = []
    prev_header = ""
    header = ""

    for line in open(fastaFileName):
        string = line.strip()
        if string[0] != ">":
            if prev_header != header:
                # print(header) # dans le cas ou on veut garde cet info
                prev_header = header
            sequence = sequence + string
        else:
            header = string
            if sequence != "":
                sequences_list.append(sequence)
                sequence = ""

    sequences_list.append(sequence)
    return sequences_list

sequencesPeaks = readFasta(peaks)
sequences1000 = readFasta(genome)


def findMotifData(sequences, motif, m, maxVar):
    """
    chercher les positions d'un motif dans un ensemble de séquence d'ADN en admettant un maximum de variations
    entrée m : taille du mot à chercher dans le motif m <= k, le meme utilise pour indexer les sequences
    entrée sequences : list contenant les sequence d'ADN
    entrée motif : chaine de caractère représentant le motif à chercher
    entrée maxVar : le maximum variations entre le motif et un mot de taille k dans la sequence
    sortie posList : list contenant les positions dans les sequences ou se trouve le motif.
    """
    count = 0; posList = []

    return posList



In [ ]:
posList = findMotifData(sequencesPeaks, motif, 4, 1)

print(len(posList))


6\. Récupérer les motifs et générer un LOGO à l'aide de l'outil WEB-LOGO https://weblogo.berkeley.edu/logo.cgi

### B -  Matrice de fréquences
7\.Nous allons implémenter la recherche de motif par matrice de fréquence. Nous allons utiliser les matrices déjà fabriquées que vous pouvez télécharger sur le site http://jaspar.genereg.net/.
Une fois que vous avez chargé votre matrice de fréquence, vous devez la transformer en matrice de probabilité en la normalisant, diviser chaque cellule par la somme de sa colonne.

In [ ]:
import numpy as np

nuc = ['A', 'C', 'G', 'T']
q = 4

#Matrice du motif Atf1, attention la matrice peut avoir plus de colonnes que le motif décrit précédemment,
#ajuster le k ou effacer les colonnes qui ne vous intéressent pas.

matFreq = np.array([[136,716,145,34,160,10,961,8,25,19],
[78,25,31,47,10,966,9,972,955,740],
[142,31,288,39,812,11,15,9,2,13],
[641,226,535,878,16,10,14,10,16,225]])

print (matFreq)

def computing_pwm(M, cols):
    """
    Calcul la matrice de poids position à partir de la matrice de frequence
    entrée M : matrice de frequence
    sortie PWM : matrice de probabilites ou poids position
    """
    PWM = M+1

    return PWM/s

k = 10
PWM = get_pwm(matrice, k); print (PWM)


8\. Déterminer les paramètres f(0)(b) du modèle nul, où
\begin{equation}
f^{(0)}(b) = \frac 1L \sum_{i=0}^{L-1} \omega_i(b)\ ,
\end{equation}

In [ ]:
def f0_calcule(PWM, L):
    """
    Calcul les valeurs de probabilites d'un modele independant de positions (modele Null)
    entrée PWM : matrice de probabilites ou poids positions
    sortie  f_0 : vecteur contenant un modele independant de positions (modele Null)
    """
    modele = []

    return modele

f_0 = f0_calcule(PWM, k)
print (f_0)

9\. Faites une fonction pour calculer log-rapport de vraisemblancee d'une sequence de taille k et la matrice poids position.
\begin{equation}
\label{eq:ll}
\ell(b_0,...,b_{k-1}) = \log_2 \frac {P(b_0,...,b_{k-1} | \omega )
}{P^{(0)}(b_0,...,b_{k-1})}
= \sum_{i=0}^{k-1} \log_2 \frac {\omega_i(b_i)}{f^{(0)}(b_i)}\ .
\end{equation}

In [ ]:
def loglikehood(seq, PWM, f_0, L):
	"""
	Calcul le rapport de vraissemblance entre une sequence et une matrice de poids position
	entrée PWM : matrice de probabilites ou poids positions
	entrée f_0 : vecteur contenant le modele independant de positions (modele Null)
	entrée seq : une sequence d'ADN de taille k, ou k est le nombre de colonnes de PWM
	sortie ll : rapport de vraissemblance
	"""
	L_0 = len(seq)
	l_win = 0

	return l_win

loglikehood('acctgcaccc'.upper(), PWM, f_0, k)

10\. Faire une fonction que pour chaque séquence cherche la/les positions avec une log-vraisemblance positive.

In [ ]:
def searchPWM(sequences, k, PWM, f_0):
    """
    Cherche les positions dans un ensemble de séquence qui maxime le rapport de vraisemblance
    entrée sequences : ensemble de séquence d'ADN
    entrée k : nombre de colonnes d'PWM
    entrée PWM : matrice de probabilités ou poids positions
    entrée f_0 : vecteur contenant le modèle indépendant de positions (modèle Null)
    sortie posList: liste contenant pour chaque séquence la/les positions ayant un rapport de vraisemblance positive
    """
    count = 0; posList = []; pos = []
    
    return posList


posList = searchPWM(sequencesPeaks, k, PWM, f_0)
print(posList)


11\.  (Bonus) Certains motifs si chevauche dans ce cas il faut choisir ceux qui ont la meilleur  log-vraisemblance. Par exemple pour une taille de motif k = 7, si vous avez deux motifs, un qui commence dans la position 45 et un autre en position 47, garder celui avec la log-vraisemblance la plus grande.

In [ ]:
def searchPWMOptmiseMotifs(sequences, k, PWM, f_0):
    """
    Cherche les positions dans un ensemble de séquence qui maxime le rapport de vraisemblance et elimine les motifs chevauchante
    entrée sequences : ensemble de séquence d'ADN
    entrée k : nombre de colonnes d'PWM
    entrée PWM : matrice de probabilités ou poids positions
    entrée f_0 : vecteur contenant le modèle indépendant de positions (modèle Null)
    sortie posList: liste contenant pour chaque séquence la/les positions ayant un rapport de vraisemblance positive

    """
    
    return None

12\.Tester l'algorithme sur vos données de chipSeq et dessiner le LOGO